In [1]:
#| include: false

import os
import pandas as pd
import numpy as np
# import plotly.express as px
import toml
# import psrc_theme
from pathlib import Path
import summary_data

# to show plotly figures in quarto HTML file
# import plotly.io as pio
# pio.renderers.default = "plotly_mimetype+notebook_connected"
# pio.templates.default = "simple_white+psrc_color" # set plotly template

config = summary_data.CONFIG
all_runs = summary_data.ALL_RUNS

- run locations

In [2]:
pd.DataFrame.from_dict(all_runs, orient='index',
                       columns=['run location'])

,run location
2023 base year,\\modelstation2\c$\Workspace\sc_23_02_17_25
2050,C:\workspace\sc_2050_base_year_2023
2050 (no telecommute),C:\workspace\sc_2050_base_year_2023_no_telecom...


## top sheet summary

In [3]:
# network summary: 'VMT','VHT','total_delay'
df_network = summary_data.load_agg_data('network/network_results.csv')
network_summary = df_network.groupby('source')[['VMT','VHT','total_delay']].sum()
network_summary = network_summary.rename(columns={'total_delay': 'Delay'})
# transit boardings
df_boardings = summary_data.load_agg_data('transit/daily_boardings_by_agency.csv')
transit_ridership = df_boardings.groupby('source')['boardings'].sum()
# mode share
df_trip = summary_data.load_agg_data('agg/dash/mode_share_county.csv')
df = df_trip.groupby('source')['trexpfac'].sum() # total trips
transit_share = df_trip.loc[df_trip['mode']=="Transit"].groupby('source')['trexpfac'].sum()/df

# emission
df_emissions = summary_data.load_agg_data('emissions/emissions_summary.csv')
df = df_emissions.loc[(df_emissions['veh_type'].isin(['light','medium','heavy'])) & \
                      (df_emissions['pollutant_name']=="CO2 Equivalent")].copy()
CO2e = df.groupby('source')['total_daily_tons'].sum()

In [4]:
run_summary2 = pd.DataFrame({
    'Transit Boardings': transit_ridership,
    '% Transit': transit_share,
    'CO2e': CO2e
})

pd.concat([network_summary,run_summary2], axis=1).style.\
        format('{:,.0f}', subset=['VMT','VHT','Delay','Transit Boardings','CO2e']).\
        format('{:.1%}', subset=['% Transit'])

,VMT,VHT,Delay,Transit Boardings,% Transit,CO2e
source,,,,,,
2023 base year,"83,284,423","2,560,336","243,164","492,670",2.2%,"41,889"
2050,"95,979,446","3,014,439","342,792","1,407,741",4.6%,"34,002"
2050 (no telecommute),"99,224,832","3,166,611","389,485","1,477,941",4.6%,"35,079"


In [5]:
# df_network['data3'].value_counts()

## population & households

In [6]:
def count_by_county(output_path: str, summary_var: str, expfac_name: str, axis_name: str):
    df = summary_data.load_agg_data(output_path)

    tab = df.groupby(['source',summary_var])[expfac_name].sum().\
        unstack(summary_var).\
        rename_axis(columns={summary_var: axis_name})
    tab['Total'] = tab.sum(axis=1)

    display(tab.style.format('{:,.0f}'))

In [7]:
count_by_county('agg/dash/pptyp_county.csv','person_county','psexpfac','Population by county')

Population by county,King,Kitsap,Outside Region,Pierce,Snohomish,Total
source,,,,,,
2023 base year,"2,303,851","275,120",8,"925,555","851,485","4,356,019"
2050,"3,071,847","356,124",281,"1,311,285","1,087,666","5,827,203"
2050 (no telecommute),"3,071,847","356,124",281,"1,311,285","1,087,666","5,827,203"


In [8]:
count_by_county('agg/dash/hh_geog.csv','hh_county','hhexpfac','Households by county')

Households by county,King,Kitsap,Pierce,Snohomish,Total
source,,,,,
2023 base year,"866,673","39,114","196,913","187,282","1,289,982"
2050,"1,121,117","42,324","255,972","225,160","1,644,573"
2050 (no telecommute),"1,121,117","42,324","255,972","225,160","1,644,573"


## employment by work location

In [9]:
count_by_county('agg/dash/person_worker_type.csv','person_work_county','psexpfac','Employment by county')

Employment by county,King,Kitsap,Outside Region,Pierce,Snohomish,Total
source,,,,,,
2023 base year,"1,306,494","84,946",nan,"289,579","284,712","1,965,731"
2050,"1,590,694","106,643",432,"424,220","371,638","2,493,627"
2050 (no telecommute),"1,631,199","108,916",416,"402,133","373,730","2,516,394"
